# Prioritizarion Playground

experimentation prior to finalization of weight of each factor

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from datetime import datetime, timedelta
import numpy as np
from scipy.stats import percentileofscore

## Getting Data

In [2]:
clients = pd.read_csv('fake_CRM_data/clients.csv')
policies = pd.read_csv('fake_CRM_data/policies.csv')

In [3]:
clients.head(3)

,client_name,client_id,account_type,address,phone_number,email,branch_id,servicing_broker_id,status,issued_on,updated_on,avg_days_between_logins,avg_user_sentiment_score
0,Christian Hartman,C001611,Personal,"831 Santiago Passage, Jenniferton, WI 58219",(479)749-2716x922,christian.hartman@gomez.com,B_US00787,S001819,Prospect,2005-05-31,2025-11-28,21.243438,7.813219
1,Elizabeth Chen,C002570,Commercial,"07375 Pena Locks Suite 248, Port Kennethhaven,...",+1-581-522-6629x482,elizabeth.chen@santiago-cobb.com,B_US00725,S005192,Active,2005-09-19,2025-11-28,9.472744,6.641544
2,Jennifer Payne,C003734,Personal,"620 Miller Trail Apt. 267, Barberhaven, KY 25108",474-347-1489x236,jennifer.payne@bailey-mathis.com,B_US00725,S009043,Inactive,2005-09-24,2025-11-28,14.717491,6.620625


In [4]:
policies.head(3)

,policy_id,client_id,line_of_business,issuing_carrier,issued_on,expiry_date,premium_amount,premium_bill_to,renewal_status,last_updated,broker_commision_pct
0,P003746,C001611,Restaurant manager,Daugherty Ltd Insurance Inc.,2005-05-31,2025-05-31,457.275285,Mortgagee,Marketing,2025-11-28,12.441244
1,P009523,C002570,"Engineer, materials",Ramsey-Zavala Insurance Inc.,2005-09-19,2025-09-19,196.490136,Subsidiary,Canceled,2025-11-28,17.969066
2,P008529,C003734,"Conservation officer, historic buildings",Reed-Chavez Insurance Inc.,2005-09-24,2025-09-24,1073.432540,Insured,Renewal Quoted,2025-11-28,5.502953


## Preprocessing

### to Datetime

In [5]:
clients.issued_on = pd.to_datetime(clients.issued_on)
clients.updated_on = pd.to_datetime(clients.updated_on)

policies.expiry_date = pd.to_datetime(policies.expiry_date)
policies.last_updated = pd.to_datetime(policies.last_updated)
policies.issued_on = pd.to_datetime(policies.issued_on)

In [6]:
clients.issued_on.dtype

dtype('<M8[ns]')

In [7]:
clients.client_id.duplicated().sum()

np.int64(0)

## Getting Scores

### time to expiry

In [8]:
today = datetime(
    year=2025,
    month=11,
    day=28
)

In [9]:
merged = pd.merge(left=clients, right=policies, how='right', on='client_id')
merged.head(3)

,client_name,client_id,account_type,address,phone_number,email,branch_id,servicing_broker_id,status,issued_on_x,updated_on,avg_days_between_logins,avg_user_sentiment_score,policy_id,line_of_business,issuing_carrier,issued_on_y,expiry_date,premium_amount,premium_bill_to,renewal_status,last_updated,broker_commision_pct
0,Christian Hartman,C001611,Personal,"831 Santiago Passage, Jenniferton, WI 58219",(479)749-2716x922,christian.hartman@gomez.com,B_US00787,S001819,Prospect,2005-05-31,2025-11-28,21.243438,7.813219,P003746,Restaurant manager,Daugherty Ltd Insurance Inc.,2005-05-31,2025-05-31,457.275285,Mortgagee,Marketing,2025-11-28,12.441244
1,Elizabeth Chen,C002570,Commercial,"07375 Pena Locks Suite 248, Port Kennethhaven,...",+1-581-522-6629x482,elizabeth.chen@santiago-cobb.com,B_US00725,S005192,Active,2005-09-19,2025-11-28,9.472744,6.641544,P009523,"Engineer, materials",Ramsey-Zavala Insurance Inc.,2005-09-19,2025-09-19,196.490136,Subsidiary,Canceled,2025-11-28,17.969066
2,Jennifer Payne,C003734,Personal,"620 Miller Trail Apt. 267, Barberhaven, KY 25108",474-347-1489x236,jennifer.payne@bailey-mathis.com,B_US00725,S009043,Inactive,2005-09-24,2025-11-28,14.717491,6.620625,P008529,"Conservation officer, historic buildings",Reed-Chavez Insurance Inc.,2005-09-24,2025-09-24,1073.432540,Insured,Renewal Quoted,2025-11-28,5.502953


In [10]:
merged['_time_to_expiry_days'] = (today - merged.expiry_date).apply(lambda x: x.days)
merged['_time_to_expiry_days']

0      181
1       70
2       65
3       57
4      167
5      254
6      353
7      240
8      349
9      118
10     262
11     112
12      87
13      85
14     130
15     222
16     221
17     278
18      23
19     281
20      44
21      26
22     112
23      21
24     339
25      66
26     135
27     329
28     305
29     211
30     290
31     284
32     281
33     305
34     110
35     242
36     197
37     126
38     144
39     318
40     166
41     281
42     109
43     143
44     250
45       5
46      77
47     273
48     230
49     364
50     355
51     210
52     340
53     148
54     116
55     235
56      41
57     128
58      19
59     330
60     207
61     216
62     313
63     347
64     143
65      23
66     128
67     329
68     276
69     320
70     277
71     132
72     255
73     346
74     181
75     188
76      27
77     118
78     127
79     234
80     199
81     362
82     315
83       2
84     289
85      47
86     165
87     239
88     221
89     328
90     213

more `time_to_expiry_score` score for less `time_to_expiry`

time_to_exiry_score = (100-percentile(time_to_expiry))/10 

score out of 10 that is a measure of how close the expiry is

In [11]:
get_time_to_expiry_score = lambda x: (100 - percentileofscore(
    a=merged['_time_to_expiry_days'],
    score=x
))/10
merged['_time_to_epxiry_score'] = merged['_time_to_expiry_days'].apply(get_time_to_expiry_score).head(3)

In [12]:
merged.head(3)

,client_name,client_id,account_type,address,phone_number,email,branch_id,servicing_broker_id,status,issued_on_x,updated_on,avg_days_between_logins,avg_user_sentiment_score,policy_id,line_of_business,issuing_carrier,issued_on_y,expiry_date,premium_amount,premium_bill_to,renewal_status,last_updated,broker_commision_pct,_time_to_expiry_days,_time_to_epxiry_score
0,Christian Hartman,C001611,Personal,"831 Santiago Passage, Jenniferton, WI 58219",(479)749-2716x922,christian.hartman@gomez.com,B_US00787,S001819,Prospect,2005-05-31,2025-11-28,21.243438,7.813219,P003746,Restaurant manager,Daugherty Ltd Insurance Inc.,2005-05-31,2025-05-31,457.275285,Mortgagee,Marketing,2025-11-28,12.441244,181,5.025
1,Elizabeth Chen,C002570,Commercial,"07375 Pena Locks Suite 248, Port Kennethhaven,...",+1-581-522-6629x482,elizabeth.chen@santiago-cobb.com,B_US00725,S005192,Active,2005-09-19,2025-11-28,9.472744,6.641544,P009523,"Engineer, materials",Ramsey-Zavala Insurance Inc.,2005-09-19,2025-09-19,196.490136,Subsidiary,Canceled,2025-11-28,17.969066,70,8.060
2,Jennifer Payne,C003734,Personal,"620 Miller Trail Apt. 267, Barberhaven, KY 25108",474-347-1489x236,jennifer.payne@bailey-mathis.com,B_US00725,S009043,Inactive,2005-09-24,2025-11-28,14.717491,6.620625,P008529,"Conservation officer, historic buildings",Reed-Chavez Insurance Inc.,2005-09-24,2025-09-24,1073.432540,Insured,Renewal Quoted,2025-11-28,5.502953,65,8.155


In [13]:
from prioritization import time_to_expiry_score, premium_at_risk_score

In [14]:
time_to_expiry_score(merged['_time_to_expiry_days'])

0      5.025
1      8.060
2      8.155
3      8.320
4      5.365
5      3.220
6      0.330
7      3.565
8      0.400
9      6.750
10     2.980
11     6.935
12     7.585
13     7.645
14     6.460
15     4.055
16     4.075
17     2.645
18     9.305
19     2.495
20     8.640
21     9.180
22     6.935
23     9.340
24     0.705
25     8.140
26     6.275
27     0.990
28     1.745
29     4.355
30     2.200
31     2.370
32     2.495
33     1.745
34     6.995
35     3.525
36     4.705
37     6.575
38     6.020
39     1.345
40     5.405
41     2.495
42     7.040
43     6.085
44     3.345
45     9.840
46     7.830
47     2.780
48     3.830
49     0.050
50     0.300
51     4.390
52     0.670
53     5.910
54     6.800
55     3.725
56     8.770
57     6.505
58     9.415
59     0.960
60     4.460
61     4.250
62     1.460
63     0.475
64     6.085
65     9.305
66     6.505
67     0.990
68     2.720
69     1.280
70     2.685
71     6.385
72     3.170
73     0.510
74     5.025
75     4.880
76     9.140

### premium Score

premium score = percentile(premium)/10

higher the premium, higher the score

In [15]:
merged['_premium_score'] = premium_at_risk_score(merged['premium_amount'])
merged[['_premium_score', 'premium_amount']].head()

,_premium_score,premium_amount
0,5.13,457.275285
1,1.14,196.490136
2,7.27,1073.432540
3,3.07,307.692029
4,4.70,407.904930


## Churn

In [16]:
from prioritization import not_churn_prob_score

pd.Series(not_churn_prob_score(merged.premium_amount, merged.avg_days_between_logins, merged.avg_user_sentiment_score))

/Users/msreeramulu/SWD/ML/Sigma/TechFest/ApexRenew/venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


0      3.591967
1      2.097220
2      3.591967
3      3.591967
4      3.591967
5      3.591967
6      3.591967
7      3.717160
8      3.591967
9      3.591967
10     0.555792
11     0.783233
12     3.591967
13     3.591967
14     3.591967
15     3.591967
16     3.591967
17     3.591967
18     3.591967
19     0.391503
20     2.097220
21     3.591967
22     3.591967
23     4.698001
24     3.591967
25     3.591967
26     3.591967
27     1.332535
28     3.591967
29     3.591967
30     2.619037
31     4.426055
32     0.561818
33     3.591967
34     3.591967
35     3.591967
36     1.332535
37     3.591967
38     3.591967
39     3.591967
40     3.591967
41     3.591967
42     3.591967
43     1.522071
44     1.332535
45     2.619037
46     3.591967
47     3.591967
48     3.591967
49     0.088210
50     3.591967
51     1.553728
52     3.591967
53     3.591967
54     3.591967
55     3.591967
56     2.860647
57     3.591967
58     3.591967
59     3.591967
60     3.591967
61     3.591967
62     2

In [ ]:
from prioritization import client_priority_GPA
client_priority_GPA(merged)